In [13]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
df = pd.read_csv('data/database.csv')
features = df[['Latitude', 'Longitude', 'Depth']]
labels = df['Magnitude']

features_in = features.as_matrix()
labels_in = labels.as_matrix()
features_in.astype(float, copy=False)
labels_in.astype(float, copy=False)

array([ 6. ,  5.8,  6.2, ...,  5.9,  6.3,  5.5])

In [15]:
features_max = np.amax(features_in, 0)
features_min = np.amin(features_in, 0)

labels_max = np.amax(labels_in, 0)
labels_min = np.amin(labels_in, 0)

feature_norm = (features_in-features_min)/(features_max-features_min)
label_norm = (labels_in-labels_min)/(labels_max-labels_min)

Xfeature = 3
Yfeature = 1
samples = 23000

features_in_reshape = np.resize(feature_norm, (samples, Xfeature))
labels_in_reshape = np.resize(label_norm, (samples, Yfeature))

In [16]:
batch_size = 20000
train_features = features_in_reshape[0:batch_size,:]
train_labels = labels_in_reshape[0:batch_size,:]

valid_size = 2500
valid_features = features_in_reshape[batch_size:batch_size+valid_size,:]
valid_labels = labels_in_reshape[batch_size:batch_size+valid_size,:]

In [17]:
# hyperparams
learning_rate = 0.001
training_iterations = 100000
display_iterations = 20000

In [18]:
X = tf.placeholder(tf.float32, shape=(None,Xfeature))
Y = tf.placeholder(tf.float32)
X

<tf.Tensor 'Placeholder_2:0' shape=(?, 3) dtype=float32>

In [7]:
L1 = 3
L2 = 3
L3 = 3

graph = tf.Graph()
with graph.as_default():

weights1 = tf.Variable(tf.random_uniform([Xfeature,L1]))
biases1 = tf.Variable(tf.constant(0.1, shape=[L1]))

weights2 = tf.Variable(tf.random_uniform([L1,L2]))
biases2 = tf.Variable(tf.constant(0.1, shape=[L2]))

weights3 = tf.Variable(tf.random_uniform([L2, L3]))
biases3 = tf.Variable(tf.constant(0.1, shape=[L3]))

weights_out = tf.Variable(tf.random_uniform([L3, Yfeature]))
biases_out = tf.Variable(tf.constant(0.1, shape=[Yfeature]))

In [8]:
layer1 = tf.matmul(X, weights1) + biases1
relu1 = tf.nn.relu(layer1)

layer2 = tf.matmul(relu1, weights2) + biases2
relu2 = tf.nn.relu(layer2)

layer3 = tf.matmul(layer2, weights2) + biases3
relu3 = tf.nn.relu(layer3)

layer_out = tf.matmul(layer3, weights_out) + biases_out

In [9]:
mean_square = tf.reduce_mean(tf.square(Y-layer_out))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

saver = tf.train.Saver()

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Initial training loss", sess.run([mean_square], feed_dict={X:train_features, Y:train_labels}))
    for i in range(training_iterations):
        sess.run([train_step], feed_dict={X:train_features,Y:train_labels})
        if i % display_iterations == 0:
            print("current loss is", sess.run([mean_square], feed_dict={X:train_features, Y:train_labels}), " at iteration ", i)
            print("validation loss is ", sess.run([mean_square], feed_dict={X:valid_features, Y:valid_labels}), " at iteration ", i)
    save_path = saver.save(sess, "./data/earthquake.ckpt")
    print("model saved")
    print("final training loss", sess.run([mean_square], feed_dict={X:train_features, Y:train_labels}))
    print("final validation loss", sess.run([mean_square], feed_dict={X:valid_features, Y:valid_labels}))

Initial training loss [1.7608527]
current loss is [1.717775]  at iteration  0
validation loss is  [1.7164772]  at iteration  0
current loss is [0.013594467]  at iteration  20000
validation loss is  [0.01444915]  at iteration  20000
current loss is [0.013594458]  at iteration  40000
validation loss is  [0.014449173]  at iteration  40000
current loss is [0.013594456]  at iteration  60000
validation loss is  [0.014449234]  at iteration  60000
current loss is [0.013594467]  at iteration  80000
validation loss is  [0.014449217]  at iteration  80000
model saved
final training loss [0.013594456]
final validation loss [0.014449218]
